In [1]:
import json
import urllib
import math
import pandas as pd 
import numpy as np

In [2]:
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 偏心率平方

In [3]:
def gcj02_to_wgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:GPS84坐标
    """
    if out_of_china(lng, lat):
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]

In [4]:
def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret


def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret

In [5]:
def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:经度
    :param lat:纬度
    :return:bool布尔值
    """
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

In [6]:
gcj02_to_wgs84(118.784129,32.041806)

[118.77892757265252, 32.0438602407582]

In [7]:
df_sts = pd.read_csv('stations.csv',encoding='gbk')

In [8]:
df_sts.head(1)

,Unnamed: 0,st_name,st_coords
0,0,双桥门北,"118.796394,32.006142"


In [9]:
df_sts['st_coords'][0]

'118.796394,32.006142'

In [10]:
float(df_sts['st_coords'][0].split(',')[0])

118.796394

In [11]:
def st_wgs84(x):
    lng=float(x.split(',')[0])
    lat=float(x.split(',')[1])
    return gcj02_to_wgs84(lng,lat)

In [12]:
df_sts['st_coords_wgs84']= df_sts['st_coords'].apply(st_wgs84)

In [13]:
df_sts.head(1)

,Unnamed: 0,st_name,st_coords,st_coords_wgs84
0,0,双桥门北,"118.796394,32.006142","[118.79121456468195, 32.00820467908147]"


In [14]:
import shapefile